In [ ]:
import pandas as pd
from math import floor, ceil
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import Image

# What is the purpose of Descriptive Analytics?

Data comes in many shapes and forms and often in large quantities. The human mind is incapable of knowing what to do with large amounts of information. 

### The purpose of Descriptive Analytics is to summarize data into insightful statistics that solve a business problem. 

# Our first sample dataset- Hotel Reservations

In [ ]:
# Reading the dataset into a pandas dataframe
reservations_df = pd.read_csv("Data/hotel_data.csv")
print(f"Imported data. There are {reservations_df.shape[0]} rows and {reservations_df.shape[1]} columns in the dataset")

In [ ]:
# Show the first few rows
reservations_df

### N.B.: Data Quality
In all data science projects, we must confirm data quality by checking such things as:
- Missing entries for each column
- Primary keys of tables
- Duplicated primary keys of tables
- Full entries duplicated
- Outliers in column values

### Data Dictionary
#### What does each column represent


Booking_ID: unique identifier of each booking

no_of_adults: Number of adults

no_of_children: Number of Children

no_of_weekend_nights: Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel

no_of_week_nights: Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel

type_of_meal_plan: Type of meal plan booked by the customer:

required_car_parking_space: Does the customer require a car parking space? (0 - No, 1- Yes)

room_type_reserved: Type of room reserved by the customer. The values are ciphered (encoded) by INN Hotels.

lead_time: Number of days between the date of booking and the arrival date

arrival_year: Year of arrival date

arrival_month: Month of arrival date

arrival_date: Date of the month

market_segment_type: Market segment designation.

repeated_guest: Is the customer a repeated guest? (0 - No, 1- Yes)

no_of_previous_cancellations: Number of previous bookings that were canceled by the customer prior to the current booking

no_of_previous_bookings_not_canceled: Number of previous bookings not canceled by the customer prior to the current booking

avg_price_per_room: Average price per day of the reservation; prices of the rooms are dynamic. (in euros)

no_of_special_requests: Total number of special requests made by the customer (e.g. high floor, view from the room, etc)

booking_status: Flag indicating if the booking was canceled or not.

# Exercise objective

Understand how cancellations are influencing overall revenue of the hotel 

# Stats 101

Statistics are scalar values that summarize a set of data observations.

Exercise: For the data observation set of "avg_price_per_room" what are some meaningful statistics?

In [ ]:
prices_df = reservations_df["avg_price_per_room"]
prices_df

## Measures of central tendency
These type of statistics summarize what the center of the dataset looks like

## Mean

Mean = sum(each_observations)/ total_number_of_observations

The mean tells us what the average of the set of data observations is. It's important to note that this measure uses the actual values of all entries in the dataset in its calculation.

In [ ]:
# Option 1: Pure python way
sum(value for value in prices_df)/sum(1 for value in prices_df)

In [ ]:
# Option 2: using pandas to write formula
prices_df.sum()/prices_df.count()

In [ ]:
# Option 3: Leveraging the function available in pandas.
prices_df.mean()

The average price of a night over the entire dataset is 103 Euros. (Kinda expensive if you ask me)

### N.B. There are many ways to code these statistics using python and pandas, we'll now focus on the most compact way (Option 3)

# Median
median =  The value of the entry located at exactly half way through the set of data observations if it were ordered

e.g. set of data observations (1,2,2,5,5): The median is the 3rd entry = 2

Note that the calculation of the median only uses the value of one of the data observations the one in the middle.

In [ ]:
prices_df.median()

## N.B. The median is a special case of quantiles where quantiles represented the entry located n-th way through the set of data observations if it were ordered.

In [ ]:
prices_df.quantile(0.5)

In [ ]:
prices_df.quantile(0.25)

# Mode

mode= The value of the entry that appears the most frequently in the set of data observations

Note that the mode, like the median, only uses the value of one of the entries in the dataset, the most common one. 

In [ ]:
prices_df.mode()[0]

# The Relationship between Mean, Median and Mode

When the mean, median and mode have all the same value, it is a sign that the data is symmetric.

N.B. This doesn't happen too often with small datasets of randome processes but as data becomes larger, they often get closer to each other.

In [ ]:
# Example of when a set of data observations has mean, median and mode equal
artificial = np.array([1,2,3,3,3,4,5])
art_df = pd.DataFrame (artificial, columns = ['value'])["value"]
mean = art_df.mean()
median = art_df.median()
mode = art_df.mode()
print(f"Mean: {mean}, Median: {median}, Mode: {mode[0]}")

# Measures of dispersion

Measure how wide the possible values of a set of data observations can be

# Range
Range = maximum_value -  minimum_value

In [ ]:
prices_df.max() - prices_df.min()

# Standard Deviation

Measures how far observations are from the mean

In [ ]:
# get the image
Image(url="images/standard_dev.png", width=800, height=800)

In [ ]:
prices_df.std()

# Interquantile measurements

IQ_measure = quantile_r - quantile_s (r>s)

In [ ]:
prices_df.quantile(0.75)-prices_df.quantile(0.25)

# Visualizing the data

# Histogram

Used to show the frequencies of continuous data

In [ ]:
plt.hist(prices_df, bins=100)
plt.show()

# Box Plot
Used to show the distribution of the data

In [ ]:
sns.boxplot(x="avg_price_per_room", orient="h", data=reservations_df, y="booking_status")

# Bar chart

Used for catgorical data (data with word values) to show their frequency

In [ ]:
sns.catplot(x='market_segment_type', data=reservations_df, hue="booking_status", kind='count')

# Analysis: Extracting insight by controlling some variables

# Who is getting these very low or free rates?

In [ ]:
# Look by market segment and number of weekend nights
freebies_df = reservations_df[reservations_df["avg_price_per_room"] == 0].copy()
sns.catplot(x='no_of_weekend_nights', data=freebies_df, hue="market_segment_type", kind='count')

Most compliementary stays are not during the weekend

In [ ]:
# Now let's create a field for total number of nights
freebies_df["total_nights"] = freebies_df["no_of_weekend_nights"]+freebies_df["no_of_week_nights"]
sns.catplot(x='total_nights', data=freebies_df, hue="market_segment_type", kind='count')

Most are for one night during the week.

In [ ]:
sns.catplot(x='total_nights', data=freebies_df, hue="room_type_reserved", kind='count')    

Most are for room type 1

# What's the average price for room type 1?

In [ ]:
room_1_no_0s = reservations_df[(reservations_df["room_type_reserved"] == "Room_Type 1") 
                & (reservations_df["avg_price_per_room"] > 0)].copy()
room_1_prices = room_1_no_0s["avg_price_per_room"]
room_1_prices.describe()

In [ ]:
sns.boxplot(x="avg_price_per_room", orient="h", data=room_1_no_0s, y="arrival_month")

In [ ]:
order = [f"2017_{a}" for a in range(7,13,1)] + [f"2018_{a}" for a in range(1,13,1)]
room_1_no_0s["year_month"] = room_1_no_0s["arrival_year"].astype("str")+"_"+room_1_no_0s["arrival_month"].astype("str")
sns.boxplot(x="avg_price_per_room", orient="h", data=room_1_no_0s, y="year_month", order=order)

Most seem to land between 90 - 110 Euros per night with Summer time being the more expensive months

# How does the price for room type differ?

In [ ]:
room_prices = reservations_df[(reservations_df["avg_price_per_room"] > 0)]
sns.boxplot(x="avg_price_per_room", orient="h", data=room_prices, y="room_type_reserved")

# How much revenue would they recover by lowering their complementary nights service?

In [ ]:
# First let's calculate the average prices for each room type , year, and month (excluding the 0s- complementary)

avg_room_prices = room_prices[["arrival_year","arrival_month","room_type_reserved", "avg_price_per_room"]]\
.groupby(["arrival_year","arrival_month","room_type_reserved",]).mean().reset_index().sort_values(by=["room_type_reserved", "arrival_year", "arrival_month"])
avg_room_prices

In [ ]:
# Next let's calculate the total number of complementary nights for each room
complementary_nights = freebies_df[["arrival_year", "arrival_month", "room_type_reserved", "total_nights"]]\
.groupby(["arrival_year","arrival_month", "room_type_reserved"]).sum().reset_index()
complementary_nights

In [ ]:
# Now let's combine the two to do our calculation
full_complementary_info =complementary_nights.merge(avg_room_prices,
                           how="left",
                           on=["arrival_year", "arrival_month", "room_type_reserved"]
                          ).fillna(room_prices["avg_price_per_room"].mean())
full_complementary_info

In [ ]:
# Now we calculate what the total cost is
full_complementary_info["potential_revenue"] = full_complementary_info["total_nights"]*full_complementary_info["avg_price_per_room"]
full_complementary_info.sort_values('potential_revenue', ascending=False)

In [ ]:
# What is the total revenue lost to complementary bookings
total_lost_rev = full_complementary_info["potential_revenue"].sum()
total_lost_rev

In [ ]:
# What is their total revenue for the period
reservations_df["total_nights"] = reservations_df["no_of_weekend_nights"]+reservations_df["no_of_week_nights"]
reservations_df["revenue"] = reservations_df["total_nights"] * reservations_df["avg_price_per_room"] 
total_revenue = reservations_df["revenue"].sum()
total_revenue

In [ ]:
# What percentage is lost to complementary bookings
total_lost_rev/total_revenue*100

# What is another source of lost revenue? Cancellations

In [ ]:
# Let's calculate what the percentage of the total revenue is lost due to cancellations
cancelled_nights = reservations_df[reservations_df["booking_status"] == "Canceled"]
cancelled_summary = cancelled_nights[["arrival_year", "arrival_month", "room_type_reserved", "total_nights"]]\
.groupby(["arrival_year","arrival_month", "room_type_reserved"]).sum().reset_index()
cancelled_summary

In [ ]:
# Now let's combine the two to do our calculation
full_cancelled_info =cancelled_summary.merge(avg_room_prices,
                           how="left",
                           on=["arrival_year", "arrival_month", "room_type_reserved"]
                          ).fillna(room_prices["avg_price_per_room"].mean())
full_cancelled_info

In [ ]:
# Now we calculate what the total cost is
full_cancelled_info["potential_revenue"] = full_cancelled_info["total_nights"]*full_cancelled_info["avg_price_per_room"]
full_cancelled_info.sort_values('potential_revenue', ascending=False)

In [ ]:
# What is the total revenue lost to cancelled bookings
total_lost_rev_cancel = full_cancelled_info["potential_revenue"].sum()
total_lost_rev_cancel

In [ ]:
# What percentage does that represent over the total revenue
total_lost_rev_cancel/total_revenue*100

# Initial findings of the Analysis
A potential of 37\% of the current total revenue is being lost due to cancelled bookings, mostly from bookings of Room_Type_1 during the peak season.

Our complementary booking program is sustainable as it currently only represents only 0.7% of revenue of the entire period.

# Assignment:

Perform the same analysis conducted for complementary bookings on cancelled bookings, present your conclusions.